# Pathfinding met Dijkstra

In deze tutorial gaan wij **grafen** bestuderen. In het bijzonder aan de hand van het pathfinding algoritme van Dijkstra. Lees als voorbereiding het **hoofdstuk 13 tm 14.1** over grafen uit de reader (https://canvas.hu.nl/courses/44572/files/5382884). Een graaf is een datastructuur waarin nodes (die in de context van grafen ook wel **vertices** heten) aan elkaar verbonden zijn door middel van **edges**. 

De manier waarop er meestal over grafen geschreven wordt is al een tuple $G = (V, E)$ waarbij G de graaf is, V de *set* van nodes/vertices en E de *set* van edges. Ik schrijf hier *set* schuingedrukt, omdat dit een bekende datastructuur voor ons is: een container waarin geen dubbelen mogen zitten. 

Vertices zijn uiteraard een classes die data kunnen bevatten, maar ook de edges kunnen soms data bevatten. Bijvoorbeeld, in een shortest path problem (zie Reader) stellen de vertices/nodes locaties voor, en bevatten dus de naam van de locatie, en de edges verbindingen tussen deze locaties. De edges bevatten de *afstand* (bijvoorbeeld in kilometers) tussen deze locaties. 

Grafen zijn een zeer flexibele manier van data structuren: zolang er maar verbindingen tussen nodes en edges gelegd kunnen worden is het een valide implementatie van een graaf. Dit kan je natuurlijk op heel veel verschillende manieren doen. Laten we daar eens een paar van gaan bekijken. 

We gaan dit graafje maken: https://en.wikipedia.org/wiki/File:Dijkstra_Animation.gif

Vermoedelijk gaat die animatie wat te snel. Hier een link waarbij je zelf kunt klikken: [Dijkstra_StapVoorStap](./Dijkstra_StapVoorStap/Dijkstra_StapVoorStap.md)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import time
import statistics as stats


In [2]:
# Een zeer generieke manier om een graaf de implementeren is er
# daarwerkelijk twee sets van te maken op basis van twee classes:
class Vertex:
    def __init__(self, identifier, data_):
        self.id = identifier
        self.data = data_

    def __eq__(self, other):  # nodig om aan een set toe te voegen
        return self.id == other.id

    def __hash__(self):  # nodig om aan een set toe te voegen
        return hash(self.id)

    def __repr__(self):
        return str(self.id) + ":" + str(self.data)


class Edge:
    def __init__(self, vertex1, vertex2, data_):
        if vertex1.id < vertex2.id:
            self.v1 = vertex1
            self.v2 = vertex2
        else:
            self.v1 = vertex2
            self.v2 = vertex1
        self.data = data_

    def __eq__(self, other):  # nodig om aan een set toe te voegen
        return self.v1.id == other.v1.id and self.v2.id == self.v2.id

    def __hash__(self):  # nodig om aan een set toe te voegen
        return hash(str(self.v1.id) + "," + str(self.v2.id))

    def __repr__(self):
        return "(" + str(self.v1.id) + "," + str(self.v2.id) + "):" + str(self.data)


class CGraph:
    def __init__(self):
        self.V = set()
        self.E = set()

    def __str__(self):
        return "V: " + str(self.V) + "\nE: " + str(self.E)


# So, for a simple shortest path problem:
gr = CGraph()
v1 = Vertex(1, "start")
v2 = Vertex(2, "")
v3 = Vertex(3, "")
v4 = Vertex(4, "")
v5 = Vertex(5, "goal")
v6 = Vertex(6, "")
gr.V.add(v1)
gr.V.add(v2)
gr.V.add(v3)
gr.V.add(v4)
gr.V.add(v5)
gr.V.add(v6)
e1 = Edge(v1, v2, 7)
e2 = Edge(v1, v3, 9)
e3 = Edge(v1, v6, 14)
e4 = Edge(v2, v3, 10)
e5 = Edge(v2, v4, 15)
e6 = Edge(v3, v4, 11)
e7 = Edge(v3, v6, 2)
e8 = Edge(v6, v5, 9)
e9 = Edge(v4, v5, 6)
gr.E.add(e1)
gr.E.add(e2)
gr.E.add(e3)
gr.E.add(e4)
gr.E.add(e5)
gr.E.add(e6)
gr.E.add(e7)
gr.E.add(e8)
gr.E.add(e9)
print(gr)

# Is dit eigenlijk al best een langdradige manier van doen...


V: {1:start, 2:, 3:, 4:, 5:goal, 6:}
E: {(2,3):10, (3,4):11, (2,4):15, (1,2):7, (1,3):9, (5,6):9, (1,6):14, (4,5):6, (3,6):2}


In [3]:
# Daarom kiezen mensen er vaak voor om een graaf makkelijker te respresenteren met standaard datatypes
# bijvoorbeeld: als we een dictionary maken met node identifiers als keys, en een tuple van (data, edges) als values
# met edges als een dictionary met vertex ids van de verbonden vertices als keys en de data van de edge als key
# krijgen we zoiets voor dezelfde graaf
DGraph = dict
gr2 = {
    1: ("start", {2: 7, 3: 9, 6: 14}),
    2: ("", {1: 7, 3: 10, 4: 15}),
    3: ("", {1: 9, 2: 10, 4: 11, 6: 2}),
    4: ("", {2: 15, 3: 11, 5: 6}),
    5: ("goal", {4: 6, 6: 9}),
    6: ("", {1: 14, 3: 2, 5: 9}),
}
print(gr2)
# Dat is makkelijker in te voeren... Maar je moet wel goed bijhouden wat ookalweer wat is.
# Bovendien heb ik er nu wat redundantie ingezet (elke node bevat de edges waaraan het verbonden zit, dus elke
# edge zit twee maal in deze datastructuur)


{1: ('start', {2: 7, 3: 9, 6: 14}), 2: ('', {1: 7, 3: 10, 4: 15}), 3: ('', {1: 9, 2: 10, 4: 11, 6: 2}), 4: ('', {2: 15, 3: 11, 5: 6}), 5: ('goal', {4: 6, 6: 9}), 6: ('', {1: 14, 3: 2, 5: 9})}


## Opgave 1: Dijkstra

Laten we eens gaan kijken hoe goed de twee mogelijke implementaties van een graaf hierboven zijn (dus die met de expliciete klasses voor Vertices en Edges, en die die de hele graaf in één dictionary gooit. 

Schrijf implementaties van Dijkstra's algoritme voor de beide implementaties van grafen (CGraph en DGraph). Tip: ga daarbij bijvoorbeeld uit van de pseudocode in de reader. Vergelijk de snelheid van beide versies van Dijkstra's algoritme door beiden meerdere keren te runnen. Vergeet niet de standaardafwijkingen te berekenen. Is één implementatie beter dan de andere? 

In [1]:

def dijkstra_cgraph(graph, begin, einde):
    # Startwaarden: alle afstanden oneindig, behalve beginpunt
    afstand = {v: float('inf') for v in graph.V}
    vorige = {v: None for v in graph.V}
    afstand[begin] = 0

    bezocht = set()
    te_bezoeken = {begin}

    while te_bezoeken:
        # Kies het dichtstbijzijnde knooppunt
        huidig = min(te_bezoeken, key=lambda v: afstand[v])
        te_bezoeken.remove(huidig)
        bezocht.add(huidig)

        if huidig == einde:
            break

        # Zoek buren van huidig knooppunt
        buren = set()
        for e in graph.E:
            if e.v1 == huidig and e.v2 not in bezocht:
                buren.add(e.v2)
            elif e.v2 == huidig and e.v1 not in bezocht:
                buren.add(e.v1)

        for buur in buren:
            te_bezoeken.add(buur)
            # Zoek de verbinding tussen huidig en buur
            for e in graph.E:
                if {e.v1, e.v2} == {huidig, buur}:
                    nieuwe_afstand = afstand[huidig] + e.data
                    if nieuwe_afstand < afstand[buur]:
                        afstand[buur] = nieuwe_afstand
                        vorige[buur] = huidig

    # Pad reconstrueren
    pad = []
    knoop = einde
    while knoop:
        pad.append(knoop)
        knoop = vorige[knoop]
    pad.reverse()

    return afstand[einde], pad


def dijkstra_dgraph(graaf, begin, einde):
    # Startwaarden
    afstand = {v: float('inf') for v in graaf}
    vorige = {v: None for v in graaf}
    afstand[begin] = 0

    bezocht = set()
    te_bezoeken = {begin}

    while te_bezoeken:
        # Kies het knooppunt met de kortste afstand
        huidig = min(te_bezoeken, key=lambda v: afstand[v])
        te_bezoeken.remove(huidig)
        bezocht.add(huidig)

        if huidig == einde:
            break

        # Verken de buren
        for buur, gewicht in graaf[huidig][1].items():
            if buur not in bezocht:
                te_bezoeken.add(buur)
                nieuwe_afstand = afstand[huidig] + gewicht
                if nieuwe_afstand < afstand[buur]:
                    afstand[buur] = nieuwe_afstand
                    vorige[buur] = huidig

    # Pad reconstrueren
    pad = []
    knoop = einde
    while knoop is not None:
        pad.append(knoop)
        knoop = vorige[knoop]
    pad.reverse()

    return afstand[einde], pad

*Schrijf hier de tekst-antwoorden voor Opgave 1*

## Opgave 2: Een andere implentatie?

Verzin (evt. samen met je teamleden) nog een andere manier om grafen te implementeren, en schrijf vervolgens weer een bijbehorende implementatie van Dijkstra's algoritme. Is deze beter dan de vorige twee manieren? Toon dit wederom aan door metingen te doen. 

Ter inspiratie: kijk eens naar **adjacency lists** https://en.wikipedia.org/wiki/Adjacency_list en **adjacency matrices** https://en.wikipedia.org/wiki/Adjacency_matrix . Zou je de edges misschien (evt. per vertex) kunnen sorteren op een manier dat Dijkstra's algoritme makkelijker maakt, en zo ja, is dat computationeel de moeite waard?

In [ ]:
edges = [
    ('A', 'B', 2),
    ('A', 'C', 5),
    ('B', 'C', 3),
    ('B', 'D', 1),
    ('C', 'D', 2)
]

# Alle unieke knopen
nodes = {'A', 'B', 'C', 'D'}


def dijkstra_edge_list(edges, nodes, start, end):
    import heapq


    buren = {node: [] for node in nodes}
    for u, v, w in edges:
        buren[u].append((v, w))

    afstanden = {node: float('inf') for node in nodes}
    vorige = {node: None for node in nodes}
    afstanden[start] = 0

    heap = [(0, start)]

    while heap:
        huidige_afstand, huidige_knoop = heapq.heappop(heap)

        if huidige_knoop == end:
            break

        for buur, gewicht in buren[huidige_knoop]:
            afstand = huidige_afstand + gewicht
            if afstand < afstanden[buur]:
                afstanden[buur] = afstand
                vorige[buur] = huidige_knoop
                heapq.heappush(heap, (afstand, buur))


    pad = []
    knoop = end
    while knoop:
        pad.append(knoop)
        knoop = vorige[knoop]
    pad.reverse()

    return afstanden[end], pad


afstand, pad = dijkstra_edge_list(edges, nodes, 'A', 'D')
print("Kortste afstand:", afstand)
print("Pad:", pad)


*Schrijf hier de tekst-antwoorden voor Opgave 2*

## Ten Slotte

Grafen kun je dus op heel veel verschillende manieren implementeren. Sommige manieren zijn beter dan anderen voor specifieke algoritmes. Let dus altijd goed op hoe je een graaf implementeert als je aan verschillende toepassingen denkt. 

Ook verschilt de implementatie uiteraard als er wel of geen data in vertices en/of edges moeten worden opgeslagen. 

In het tweede grote practicum, zullen we kijken naar een coordination graph. Dit is een graaf waarbij elke vertex een beslisvariabele voorstelt met een beperkt aantal mogelijke beslissingen (acties / waarden). Elke edge representeert welke lokale beloning je krijgt voor de mogelijke beslissingen voor de beslisvariabelen die dezen verbindt. Kijk bij dit practicum eerst eens goed hoe de graaf geïmplementeerd is. Mocht je daar vragen over hebben stel ze dan z.s.m. aan een van de docenten of aan de studentassistent.   